In [77]:
import http.client

import os
from dotenv import load_dotenv
from openai import OpenAI
from pathlib import Path
from urllib.parse import urlencode

In [87]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
rapid_api_key = os.getenv('RAPIDAPI_KEY')

if not api_key or not rapid_api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key and rapid_api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API keys found and looks good so far!")

API keys found and looks good so far!


In [17]:
def load_resume():
    try:
        file_path = Path("./data/Resume.txt")
        if not file_path.exists():
            raise FileNotFoundError("Resume.txt file not found in the data directory.")
        content = file_path.read_text(encoding="utf-8")
        return content
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except UnicodeDecodeError:
        print("Error: Could not decode file. Try a different encoding (e.g., 'latin-1').")
    except Exception as e:
        print(f"Unexpected error: {e}")

In [45]:
system_prompt = """
You are a job finding assistant. Based on the resume provided, I want you to extract the following information, this information actually goes into a path param for a job board API, hence the format requiment is important:
1. field: title_filter
description: String
Search on the job title. For example: Software Engineer, Data Scientist, Product Manager.
Extract a tuple of relevant job titles for the user from the resume provided.

2. field: location_filter
description: Filter on location. Please do not search on abbreviations like US, UK, NYC. Instead, search on full names like United States, New York, United Kingdom.

You may filter on more than one location in a single API call using the OR parameter. For example: Dubai OR Netherlands OR Belgium.

Extract this from the resume provided and the past places the person has worked or lived.

3. field: ai_experience_level_filter
description: String
BETA Feature. Filter on a certain required experience level as identified by our AI, the options are:

0-2/2-5/5-10/10+

To filter on more than one job type, please delimit by comma with no space, like such: 0-2,2-5

Extract this from the resume provided based on years of experience mentioned in the resume. Calculate the total based on experience mentioned for each company.
"""

In [46]:
user_prompt = f"""
Resume provided, extract the fields required and return only valid **JSON** matching the exact schema:
Schema:
    "advanced_title_filter" : "string",
    "location_filter" : "string",
    "ai_experience_level_filter" : "string"
Resume:
{load_resume()}"""

In [47]:
openai = OpenAI()

In [48]:
response_schema = {
    "advanced_title_filter" : "string",
    "location_filter" : "string",
    "ai_experience_level_filter" : "string"
}

In [49]:
def messages_for():
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

def summarize():
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(),
        response_format = {
            "type": "json_object"
        }
    )
    return json.loads(response.choices[0].message.content)

In [50]:
ai_response = summarize()

In [51]:
print(ai_response)

{'advanced_title_filter': 'Senior Software Developer, Software Development Engineer II, Software Engineer, Machine Learning Engineer', 'location_filter': 'Bangalore OR Gurugram', 'ai_experience_level_filter': '5-10'}


In [68]:
# print(ai_response)
fields = ai_response
title_filter = fields["advanced_title_filter"].split(",")
location_filter = fields["location_filter"]
experience_level_filter = fields["ai_experience_level_filter"]
print(title_filter)
print(location_filter)
print(experience_level_filter)

for title in title_filter:
    print(f"Title: {title.strip()}")

['Senior Software Developer', ' Software Development Engineer II', ' Software Engineer', ' Machine Learning Engineer']
Bangalore OR Gurugram
5-10
Title: Senior Software Developer
Title: Software Development Engineer II
Title: Software Engineer
Title: Machine Learning Engineer


In [72]:
job_response_list = []
for title in title_filter:
    job_api_request_params = {
        "limit": 10,
        "offset": 0,
        "title_filter": title.strip(), # Get titles from AI
        "location_filter": location_filter, # Get location from AI
        "ai_employment_type_filter": "FULL_TIME",
        "ai_experience_level_filter": experience_level_filter, # 0-2/2-5/5-10/10+ from AI
        "ai_visa_sponsorship_filter": False
    }

    headers = {
        'x-rapidapi-key': rapid_api_key,
        'x-rapidapi-host': "active-jobs-db.p.rapidapi.com"
    }

    params = urlencode(job_api_request_params)
    conn = http.client.HTTPSConnection("active-jobs-db.p.rapidapi.com")
    conn.request("GET", "/active-ats-7d?" + params, headers=headers)

    res = conn.getresponse()
    data = res.read()

    print(type(data.decode("utf-8")))
    print(data.decode("utf-8"))
    job_response_list.append(data.decode("utf-8"))

<class 'str'>
[{"id":"1907754604","date_posted":"2025-11-28T00:00:00","date_created":"2025-11-28T12:39:27.407929","title":"Senior Software Developer in Test","organization":"Q2","organization_url":"","date_validthrough":"2025-12-31T00:00:00","locations_raw":[{"@type":"Place","address":{"@type":"PostalAddress","addressCountry":"India","addressLocality":"Bangalore, India"}}],"locations_alt_raw":["Bangalore, India"],"location_type":null,"location_requirements_raw":null,"salary_raw":null,"employment_type":["FULL_TIME"],"url":"https://q2ebanking.wd5.myworkdayjobs.com/Q2/job/Bangalore-India/Senior-Software-Developer-in-Test_REQ-11519","source_type":"ats","source":"workday","source_domain":"q2ebanking.wd5.myworkdayjobs.com","organization_logo":null,"cities_derived":["Bengaluru"],"counties_derived":["Bangalore North"],"regions_derived":["Karnataka"],"countries_derived":["India"],"locations_derived":["Bengaluru, Karnataka, India"],"timezones_derived":["Asia/Kolkata"],"lats_derived":[12.9767936]

In [85]:
# Step 1: Debug your input data first
list_of_strings = job_response_list
import json
import pandas as pd

# Parse all jobs (using the working nuclear approach)
all_jobs = []
for job_str in list_of_strings:
    try:
        jobs = json.loads(job_str)
        for job in jobs:
            # Pre-process lists for CSV compatibility
            job['employment_type'] = ', '.join(str(x) for x in job.get('employment_type', []))
            job['locations_derived'] = ', '.join(str(x) for x in job.get('locations_derived', []))
            all_jobs.append(job)
    except:
        continue

# Create DataFrame
df = pd.DataFrame(all_jobs)

# Select key columns for CSV
csv_columns = [
    'id', 'title', 'organization', 'employment_type', 'locations_derived',
    'date_posted', 'date_validthrough', 'url', 'source', 'domain_derived'
]

df_csv = df[csv_columns].fillna('')

# Save to CSV (Excel-friendly)
df_csv.to_csv('job_listings_bengaluru.csv', index=False)
print("✅ Saved to 'job_listings_bengaluru.csv'")

# Also save with full details
df.to_csv('job_listings_full.csv', index=False)
print("✅ Saved full details to 'job_listings_full.csv'")

# Preview
print(f"\n📊 {len(df)} jobs saved:")
print(df_csv[['title', 'organization', 'url']].head().to_string(index=False))


✅ Saved to 'job_listings_bengaluru.csv'
✅ Saved full details to 'job_listings_full.csv'

📊 24 jobs saved:
                                     title  organization                                                                                                                   url
         Senior Software Developer in Test            Q2           https://q2ebanking.wd5.myworkdayjobs.com/Q2/job/Bangalore-India/Senior-Software-Developer-in-Test_REQ-11519
         Senior Software Developer in Test        Deluxe https://bydeluxe.wd5.myworkdayjobs.com/Deluxe_External/job/IN-KA-Bangalore/Senior-Software-Developer-in-Test_R0015745
                 Senior Software Developer     Quantiphi     https://quantiphi.wd1.myworkdayjobs.com/Careers_at_Quantiphi/job/IN-KA-Bengaluru/Senior-Software-Developer_JR7928
Software Development Engineer II (Backend) AppsForBharat                                                                      https://careers.kula.ai/appsforbharat/jobs/19566
  Senior Software E